In [34]:
%load_ext autoreload

from pathlib import Path

import numpy as np

import skimage
import skimage.feature

from bokeh.plotting import output_notebook, figure, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider

import fish

output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [10]:
THIS_DIR = Path.cwd()
ROOT_DIR = THIS_DIR.parent
DATA_DIR = ROOT_DIR / 'data'

In [11]:
raw_movie = fish.load_or_read(DATA_DIR / 'drug.avi')

DEBUG:fish.io:Loaded frames from C:\Users\joshk\projects\fish\data\drug.npy


In [12]:
movie = np.fliplr(raw_movie)
print(movie.shape)

(1862, 1024, 1280)


In [13]:
def show_frame(frame):
    W = frame.shape[1]
    H = frame.shape[0]

    def _(doc):
        p = figure(match_aspect = True)
        p.x_range.range_padding = p.y_range.range_padding = 0

        source = ColumnDataSource(data=dict(image=[frame[:, :]]))

        p.image(image='image', source = source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")

        doc.add_root(p)
        
    return show(_)

def show_frames(frames):
    N = frames.shape[0]
    W = frames.shape[2]
    H = frames.shape[1]

    def _(doc):
        p = figure(match_aspect = True)
        p.x_range.range_padding = p.y_range.range_padding = 0

        source = ColumnDataSource(data=dict(image=[frames[0, :, :]]))

        slider = Slider(start=0, end=(N-1), value=0, step=1, title="Frame", callback_policy='throttle', callback_throttle=100)
        
        def update(attr, old, new):
            source.data = dict(image=[frames[new, :, :]])
        
        slider.on_change('value_throttled', update)

        p.image(image='image', source = source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")

        doc.add_root(column(p, slider))
        
    return show(_)

In [14]:
show_frames(movie)
show_frame(movie[0])

In [66]:
def canny(frames):
    N = frames.shape[0]
    W = frames.shape[2]
    H = frames.shape[1]

    def _(doc):
        orig = figure(match_aspect = True)
        cann = figure(match_aspect = True)
        orig.x_range.range_padding = orig.y_range.range_padding = 0
        cann.x_range = orig.x_range
        cann.y_range = orig.y_range
        
        frame_opts = {
            'index': 0
        }
        canny_opts = {
            'sigma': 1,
            'low_threshold': 10,
            'high_threshold': 200,
        }

        orig_source = ColumnDataSource(data=dict(image=[frames[frame_opts['index'], :, :]]))
        cann_source = ColumnDataSource(data=dict(image=[skimage.feature.canny(frames[frame_opts['index'], :, :], **canny_opts)]))

        frame_slider = Slider(start=0, end=(N-1), value=frame_opts['index'], step=1, title="Frame", callback_policy='mouseup', callback_throttle=100)
        sigma_slider = Slider(start=0, end=10, value=canny_opts['sigma'], step=.01, title="Sigma", callback_policy='mouseup', callback_throttle=100)
        low_slider = Slider(start=0, end=255, value=canny_opts['low_threshold'], step=.01, title="Low Threshold", callback_policy='mouseup', callback_throttle=100)
        high_slider = Slider(start=0, end=255, value=canny_opts['high_threshold'], step=.01, title="High Threshold", callback_policy='mouseup', callback_throttle=100)
        
        def updater(opt_dict, key):
            def _(attr, old, new):
                opt_dict[key] = new
                
                f = frames[frame_opts['index']]
                c = skimage.feature.canny(f, **canny_opts)
                orig_source.data = dict(image=[f])
                cann_source.data = dict(image=[c])
                
            return _
        
        frame_slider.on_change('value_throttled', updater(frame_opts, 'index'))
        sigma_slider.on_change('value_throttled', updater(canny_opts, 'sigma'))
        low_slider.on_change('value_throttled', updater(canny_opts, 'low_threshold'))
        high_slider.on_change('value_throttled', updater(canny_opts, 'high_threshold'))

        orig.image(image='image', source = orig_source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")
        cann.image(image='image', source = cann_source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")

        doc.add_root(column(row(orig, cann), frame_slider, row(sigma_slider, low_slider, high_slider)))
        
    return show(_)

canny(movie)